<a href="https://colab.research.google.com/github/xiscoding/utils/blob/main/GhidraSnippets_HackOvert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://github.com/HackOvert/GhidraSnippets?tab=readme-ov-file#dumping-refined-pcode

# Working with the Flat APIs
As stated in the introduction, the simplest method of using Ghidra's API is via the Flat APIs. As of Ghidra 10.2 this includes FlatProgramAPI, FlatDecompilerAPI, and the most recent FlatDebuggerAPI. The Flat APIs are designed to offer a stable API interface to some of Ghidra's high level functionality. The simplicity and convenience offered by the Flat APIs can be quickly eclipsed by their limited functionality. However, if what you need to do is offered via the Flat APIs, it's highly recommended you use them.

## Using the FlatProgramAPI
The FlatProgramAPI is a simple interface to Ghidra's program related functionality. This includes functions, data, instructions, etc.

In [ ]:
from ghidra.program.flatapi import FlatProgramAPI

state = getState()
program = state.getCurrentProgram()
fpapi = FlatProgramAPI(program)

for x in dir(fpapi): print(x)
print(fpapi.currentProgram)
print(fpapi.firstFunction)

## Using the FlatDecompilerAPI
The FlatDecompilerAPI is a simple interface to Ghidra's decompiler and requires an instance of the FlatProgramAPI for initialization in order to do anything useful. In other words, we need a target program to use the decompiler.

In [ ]:
from ghidra.app.decompiler.flatapi import FlatDecompilerAPI
from ghidra.program.flatapi import FlatProgramAPI

fpapi = FlatProgramAPI(getState().getCurrentProgram())
fdapi = FlatDecompilerAPI(fpapi)

for x in dir(fdapi): print(x)

main_decomp = fdapi.decompile(fpapi.getFunction('main'))
print(main_decomp)


## Using the FlatDebuggerAPI
The FlatDebuggerAPI is a simple interface to Ghidra's debugger and trace functionality. This is a new feature as of Ghidra 10.2 and yet to be documented in the Ghidra 10.2 API docs. For some extra context about this API, see DemoDebuggerScript.java. As I learn more about this API I'll update this section.

In [ ]:
from ghidra.debug.flatapi import FlatDebuggerAPI

fdapi = FlatDebuggerAPI

for x in dir(fdapi): print(x)

# Working with Projects
A Ghidra Project (class GhidraProject) contains a logical set of program binaries related to a reverse engineering effort. Projects can be shared (collaborative) or non-shared (private). The snippets in this section deal with bulk import and analysis, locating project files on disk, and more.

## Get the name and location on disk of the current project
If you're looking to automate analysis using headless scripts you'll likley have to deal with project management. This includes adding binaries to existing projects, cleaning up old projects, or perhaps syncing analysis to shared projects.

In [ ]:
state = getState()
project = state.getProject()
program = state.getCurrentProgram()
locator = project.getProjectData().getProjectLocator()
print("type(state):           {}".format(type(state)))
print("type(project):         {}".format(type(project)))
print("type(program):         {}".format(type(program)))
print("type(locator):         {}".format(type(locator)))
print("Project Name:          {}".format(locator.getName()))
print("Files in this project: {}".format(project.getProjectData().getFileCount()))
print("Is a remote project:   {}".format(locator.isTransient()))
print("Project location:      {}".format(locator.getLocation()))
print("Project directory:     {}".format(locator.getProjectDir()))
print("Lock file:             {}".format(locator.getProjectLockFile()))
print("Marker file:           {}".format(locator.getMarkerFile()))
print("Project URL:           {}".format(locator.getURL()))

## List all programs in the current project
A Ghidra project is a logical collection binaries that relate to a specific RE effort. This might be a single executable with multiple shared objects, or multiple executables with numerous third-party libraries, kernel modules, and drivers.

In [ ]:
state = getState()
project = state.getProject()
locator = project.getProjectData().getProjectLocator()
projectMgr = project.getProjectManager()
activeProject = projectMgr.getActiveProject()
projectData = activeProject.getProjectData()
rootFolder = projectData.getRootFolder()

print("type(state):           {}".format(type(state)))
print("type(project):         {}".format(type(project)))
print("type(projectMgr):      {}".format(type(projectMgr)))
print("type(activeProject):   {}".format(type(activeProject)))
print("type(projectData):     {}".format(type(projectData)))
print("type(rootFolder):      {}".format(type(rootFolder)))

projectName = locator.getName()
fileCount = projectData.getFileCount()
files = rootFolder.getFiles()

print("The project '{}' has {} files in it:".format(projectName, fileCount))
for file in files:
	print("\t{}".format(file))

# Working with Programs
A Program is a binary component within a Ghidra Project. The snippets in this section deal with gathering information about the Programs within a Project.

## List the current program name and location on disk

In [ ]:
state = getState()
currentProgram = state.getCurrentProgram()
name = currentProgram.getName()
location = currentProgram.getExecutablePath()
print("The currently loaded program is: '{}'".format(name))
print("Its location on disk is: '{}'".format(location))

## List the name and size of program sections

In [ ]:
blocks = currentProgram.getMemory().getBlocks()
for block in blocks:
	print("Name: {}, Size: {}".format(block.getName(), block.getSize()))

# Working with Functions
A Function is a subroutine within an Program. The snippets in this section deal with gathering information about Functions and modifying them within an Program.

## Enumerate all functions printing their name and address
There are at least two ways to do this. The output is the same for each method.

In [ ]:
# Method 1:
func = getFirstFunction()
while func is not None:
    print("Function: {} @ 0x{}".format(func.getName(), func.getEntryPoint()))
    func = getFunctionAfter(func)

# Method 2:
fm = currentProgram.getFunctionManager()
funcs = fm.getFunctions(True) # True means 'forward'
for func in funcs:
    print("Function: {} @ 0x{}".format(func.getName(), func.getEntryPoint()))

## Get a function name by address
This snippet will help you correlate addresses with associated function names.

In [ ]:
# helper function to get a Ghidra Address type
def getAddress(offset):
    return currentProgram.getAddressFactory().getDefaultAddressSpace().getAddress(offset)

# get a FunctionManager reference for the current program
functionManager = currentProgram.getFunctionManager()

# getFunctionAt() only works with function entryPoint addresses!
# returns `None` if address is not the address of the first
# instruction in a defined function. Consider using
# getFunctionContaining() method instead.
addr = getAddress(0x00100690)
funcName = functionManager.getFunctionAt(addr).getName()
print(funcName)

# check if a specific address resides in a function
addr = getAddress(0x00100691)
print(functionManager.isInFunction(addr))

# get the function an address belongs to, returns `None` if the address
# is not part of a defined function.
addr = getAddress(0x00100691)
print(functionManager.getFunctionContaining(addr))

## Get a function address by name
Have a name for a function but want the entry point address for it? This will help you do that. Just remember that two or more functions can share the same name (due to function overloading), so Ghidra will return an array (Python list) you have to consider iterating over.

In [ ]:
# Note that multiple functions can share the same name, so Ghidra's API
# returns a list of `Function` types. Just keep this in mind.
name = "main"
funcs = getGlobalFunctions(name)
print("Found {} function(s) with the name '{}'".format(len(funcs), name))
for func in funcs:
	print("{} is located at 0x{}".format(func.getName(), func.getEntryPoint()))

## Get cross references to a function
Ghidra makes it easy to find all cross references to a function using getReferencesTo. To use this, you'll just need the function's entry address which can be acquired using the getEntryPoint method on a function object. Let's take a look at an example where we find all cross references to functions named "system".

In [ ]:
fm = currentProgram.getFunctionManager()
funcs = fm.getFunctions(True)
for func in funcs:
  if func.getName() == "system":
    print("\nFound 'system' @ 0x{}".format(func.getEntryPoint()))
    entry_point = func.getEntryPoint()
    references = getReferencesTo(entry_point)
    for xref in references:
      print(xref)

## Analyzing function call arguments
This snippet uses a TARGET_ADDR which should be the address of a call to return the call arguments at that address. Thanks to gipi for suggesting this much cleaner way to obtain function call arguments than previously listed!

In [ ]:
from ghidra.app.decompiler import DecompileOptions
from ghidra.app.decompiler import DecompInterface
from ghidra.util.task import ConsoleTaskMonitor

# # Disassembly shows: 00434f6c    CALL    FUN_00433ff0
# # Decompiler shows:  uVar1 = FUN_00433ff0(param_1,param_2,param_3);
TARGET_ADDR = toAddr(0x00434f6c)

options = DecompileOptions()
monitor = ConsoleTaskMonitor()
ifc = DecompInterface()
ifc.setOptions(options)
ifc.openProgram(currentProgram)

func = getFunctionContaining(TARGET_ADDR)
res = ifc.decompileFunction(func, 60, monitor)
high_func = res.getHighFunction()
pcodeops = high_func.getPcodeOps(TARGET_ADDR)
op = pcodeops.next()
print(op.getInputs())

## Analyzing function call arguments at cross references
In this snippet we locate cross references to a target function (TARGET_FUNC) and show how we can analyze the arguments passed to each call. This can be helpful in analyzing malware, or potentially vulnerable functions. For malware analysis, this may help "decrypt" strings, or in vulnerability research this may help locate functions that may be vulnerable if called with an incorrect value. The specific analysis performed on the arguments of a called target function are up to you. This snippet will allow you to add your own analysis as you see fit.

In [ ]:
from ghidra.app.decompiler import DecompileOptions
from ghidra.app.decompiler import DecompInterface
from ghidra.util.task import ConsoleTaskMonitor

TARGET_FUNC = "FUN_62de9540"

# Step 1. Get functions that call the target function ('callers')
target_addr = 0
callers = []
funcs = getGlobalFunctions(TARGET_FUNC)
for func in funcs:
    if func.getName() == TARGET_FUNC:
        print("\nFound {} @ 0x{}".format(TARGET_FUNC, func.getEntryPoint()))
        target_addr = func.getEntryPoint()
        references = getReferencesTo(target_addr)
        for xref in references:
            call_addr = xref.getFromAddress()
            caller = getFunctionContaining(call_addr)
            callers.append(caller)
        break

# deduplicate callers
callers = list(set(callers))

# Step 2. Decompile all callers and find PCODE CALL operations leading to `target_add`
options = DecompileOptions()
monitor = ConsoleTaskMonitor()
ifc = DecompInterface()
ifc.setOptions(options)
ifc.openProgram(currentProgram)

for caller in callers:
    res = ifc.decompileFunction(caller, 60, monitor)
    high_func = res.getHighFunction()
    lsm = high_func.getLocalSymbolMap()
    symbols = lsm.getSymbols()
    if high_func:
        opiter = high_func.getPcodeOps()
        while opiter.hasNext():
            op = opiter.next()
            mnemonic = str(op.getMnemonic())
            if mnemonic == "CALL":
                inputs = op.getInputs()
                addr = inputs[0].getAddress()
                args = inputs[1:] # List of VarnodeAST types
                if addr == target_addr:
                    print("Call to {} at {} has {} arguments: {}".format(addr, op.getSeqnum().getTarget(), len(args), args))
                    for arg in args:
                        # Do stuff with each `arg` here...
                        # Not sure what to do? Check out this great article by Lars A. Wallenborn for some ideas:
                        # https://blag.nullteilerfrei.de/2020/02/02/defeating-sodinokibi-revil-string-obfuscation-in-ghidra/
                        # Specifically, search for the function implementation of "traceVarnodeValue"
                        pass

## Rename functions based on strings
In some cases strings will either hint at or give the name of a function when symbols aren't available. In these cases, we can rename functions based on these strings to help perform reverse engineering. This becomes a daunting task when there are hundreds or thousands of these cases, making the process of copy, rename, paste, a soul crushing task. The power of tools like Ghidra is that you can script this. Take for example this decompiled function found in a binary:
```
undefined4 FUN_00056b58(void)
{
	undefined4 in_r3;

	register_function(1, "core_Init_Database", FUN_00058294);
	register_function(1, "core_Clear_Database", FUN_00058374);
	register_function(1, "core_Auth_Database", FUN_00058584);
	register_function(1, "core_Add_User_Database", FUN_00058650);

	// ... hundreds more in this function and in others ...

	return in_r3;
}
```
In this function, we have calls to a `register_function` which correlates an event string with a handler function. We want to rename these functions so that FUN_00058294 becomes `core_Init_Database` and so on. Below is code that performs this task for `every register_function` in the target binary.

In [ ]:
from ghidra.app.decompiler import DecompileOptions
from ghidra.app.decompiler import DecompInterface
from ghidra.util.task import ConsoleTaskMonitor

def getString(addr):
	mem = currentProgram.getMemory()
	core_name_str = ""
	while True:
		byte = mem.getByte(addr.add(len(core_name_str)))
		if byte == 0:
			return core_name_str
		core_name_str += chr(byte)

# Get decompiler interface
options = DecompileOptions()
monitor = ConsoleTaskMonitor()
ifc = DecompInterface()
ifc.setOptions(options)
ifc.openProgram(currentProgram)

# Get reference to `register_function`
fm = currentProgram.getFunctionManager()
funcs = fm.getFunctions(True)
register_function = None
for func in funcs:
	if func.getName() == "register_function":
		register_function = func
		break

# Get xrefs to "register_function"
entry_point = register_function.getEntryPoint()
xrefs = getReferencesTo(entry_point)
callers = []
for xref in xrefs:
	from_addr = xref.getFromAddress()
	caller = fm.getFunctionContaining(from_addr)
	if caller not in callers:
		callers.append(caller)

# Process callers (functions calling `register_function`)
for caller in callers:
	if not caller:
		continue
	res = ifc.decompileFunction(caller, 60, monitor)
	hf = res.getHighFunction()
	opiter = hf.getPcodeOps()
	while opiter.hasNext():
		op = opiter.next()
		mnemonic = op.getMnemonic()
		if mnemonic == "CALL":
			call_target = op.getInput(0)
			if call_target.getAddress() == entry_point:
				core_name = op.getInput(2)
				core_func = op.getInput(3)
				core_name_def = core_name.getDef()
				core_name_addr = toAddr(core_name_def.getInput(0).getOffset())
				core_string = getString(core_name_addr)
				core_func_addr = toAddr(core_func.getDef().getInput(1).getOffset())
				core_func_obj = fm.getFunctionAt(core_func_addr)
				core_func_obj.setName(core_string, ghidra.program.model.symbol.SourceType.DEFAULT)

# Working with Instructions
## Print all instructions in a select function
Just like objdump or a disas command in GDB, Ghidra provides a way to dump instructions if you need. You might do this to generate input for another application, or for documenting issues found during analysis. Whatever you use case might be, you can easily acquire the address, opcodes, and instruction text for a target function, or specific addresses.

In [ ]:
from binascii import hexlify

listing = currentProgram.getListing()
main_func = getGlobalFunctions("main")[0] # assume there's only 1 function named 'main'
addrSet = main_func.getBody()
codeUnits = listing.getCodeUnits(addrSet, True) # true means 'forward'

for codeUnit in codeUnits:
	print("0x{} : {:16} {}".format(codeUnit.getAddress(), hexlify(codeUnit.getBytes()), codeUnit.toString()))

# Find all calls and jumps to a register
This snippet shows how to find all instructions in an x86 program that are calls or jumps to a regitser. This information can be useful when attempting to track down a crash by researching code flow in hard-to-debug targets. `getRegister` returns `None` when the specified index is not a regitser. We use `startswith('J')` to account for all jump variants. This is not architecture agnostic and a little goofy but it gets the job done.

In [ ]:
listing = currentProgram.getListing()
func = getFirstFunction()
entryPoint = func.getEntryPoint()
instructions = listing.getInstructions(entryPoint, True)

for instruction in instructions:
    addr = instruction.getAddress()
    oper = instruction.getMnemonicString()
    if (oper.startswith('CALL') or oper.startswith('J') and instruction.getRegister(0)):
        print("0x{} : {}".format(addr, instruction))

# Count all mnemonics in a binary
While recently preparing to teach some introductary x86, I wanted to know the most used mnemonics appearing in a given application to make sure I covered them. This is insanely easy to do in Binary Ninja, but a bit more involved in Ghidra. Essentially, we track mnemonics in a dictionary and increment the count as we process all instructions in a binary.

This requires getting a `InstructionDB` and using the `getMnemonicString` method to determine the mnemonic of the native assembly instruction. At the end of this snippet, we copy/pasta code from StackOverflow to sort our collected data without really thinking about how it works and we call it a day. All joking aside, this is a pretty neat way to prioritize which instructions you should focus on learning if you're learning a new architecture and don't know where to begin.

In [ ]:
instructions = {}
af = currentProgram.getAddressFactory()

func = getFirstFunction()
addr = af.getAddress(str(func.getEntryPoint()))
ins = getInstructionAt(addr)

while ins is not None:
    mnemonic = ins.getMnemonicString()
    if mnemonic in instructions:
        instructions[mnemonic] += 1
    else:
        instructions[mnemonic] = 1
    ins = ins.getNext()

ins_sorted = [ (i,mnem) for mnem,i in instructions.iteritems() ]
ins_sorted.sort(reverse=True)
for i,mnem in ins_sorted:
    print("{}: {}".format(i, mnem))

# Working with Variables
## Get a stack variable from a Varnode or VarnodeAST
When working with refined PCode you'll almost exclusively be dealing with VarnodeAST or PCodeOpAST objects. Correlating these objects to stack variables is not something exposed by the Ghidra API (as far as I can tell in v9.2.2). This leads to a complex mess of taking a varnode and comparing it to the decompiler's stack variable symbols for a given function. It's not intutitive, and quite frankly, it's been the most confusing and complex thing I've done with the Ghidra API to date.

This function works when you're passing a Varnode/AST of a simple variable, say something like this:
```
memset(local_88,0,0x10);
```
If you want to know what that the first argument (local_88) is named "local_88" and get its size, you can use this function. If that first parameter include nested `PCodeOpAST`s however, say something like (char *)local_a8, this function will not work because the variable is "wrapped" inside of a CAST operation. In order to work, this needs to be paired with `get_vars_from_varnode` (ctrl-f to find its definition) to unwrap this onion, isolate the variable VarnodeAST, and then pass it here. There's an example of doing that in this document under the heading "Get stack variables from a PcodeOpAST".

In [ ]:
def get_stack_var_from_varnode(func, varnode):
    if type(varnode) not in [Varnode, VarnodeAST]:
        raise Exception("Invalid value. Expected `Varnode` or `VarnodeAST`, got {}.".format(type(varnode)))

    bitness_masks = {
        '16': 0xffff,
        '32': 0xffffffff,
        '64': 0xffffffffffffffff,
    }

    try:
      addr_size = currentProgram.getMetadata()['Address Size']
      bitmask = bitness_masks[addr_size]
    except KeyError:
      raise Exception("Unsupported bitness: {}. Add a bit mask for this target.".format(addr_size))

    local_variables = func.getAllVariables()
    vndef = varnode.getDef()
    if vndef:
        vndef_inputs = vndef.getInputs()
        for defop_input in vndef_inputs:
            defop_input_offset = defop_input.getAddress().getOffset() & bitmask
            for lv in local_variables:
                unsigned_lv_offset = lv.getMinAddress().getUnsignedOffset() & bitmask
                if unsigned_lv_offset == defop_input_offset:
                    return lv

        # If we get here, varnode is likely a "acStack##" variable.
        hf = get_high_function(func)
        lsm = hf.getLocalSymbolMap()
        for vndef_input in vndef_inputs:
            defop_input_offset = vndef_input.getAddress().getOffset() & bitmask
            for symbol in lsm.getSymbols():
                if symbol.isParameter():
                    continue
                if defop_input_offset == symbol.getStorage().getFirstVarnode().getOffset() & bitmask:
                    return symbol

    # unable to resolve stack variable for given varnode
    return None

## Get stack variables from a PcodeOpAST
If you took a look at the code under the section "Get a stack variable from a Varnode or VarnodeAST", you'll probably be asking why that code works for something like: `memset(local_88,0,0x10);` but it fails for strchr((char *)local_a8,10);. The reason is that local_88 is a VarnodeAST while (char *)local_a8 is a PcodeOpAST. In other words, the local_a8 varnode is "wrapped" inside of a PcodeOpAST and you can't associate it to any kind of meaningful value without first "unwrapping" it. Of course, a wrapped VarnodeAST could be wrapped in numerous CAST operations and INT_ADD operations, etc. So how do we handle this? Recursion. * shudder *.

Fair warning, recursion is my computer science nemisis. If you look at this code and think "this is odd" - you're probably right!

That being said, let's slap something like (char *)local_a8 into this function and see if we can get the associated variable name(s) out of it. It's perfectly fine to pass a PcodeOpAST with multiple variables (e.g. (long)iVar2 + local_a0) into this function. It will just return a list of all variables contained in that VarnodeAST.



In [ ]:
def get_vars_from_varnode(func, node, variables=None):
    if type(node) not in [PcodeOpAST, VarnodeAST]:
        raise Exception("Invalid value passed. Got {}.".format(type(node)))

    # create `variables` list on first call. Do not make `variables` default to [].
    if variables == None:
        variables = []

    # We must use `getDef()` on VarnodeASTs
    if type(node) == VarnodeAST:
        # For `get_stack_var_from_varnode` see:
        # https://github.com/HackOvert/GhidraSnippets
        # Ctrl-F for "get_stack_var_from_varnode"
        var = get_stack_var_from_varnode(func, node)
        if var and type(var) != HighSymbol:
            variables.append(var.getName())
        node = node.getDef()
        if node:
            variables = get_vars_from_varnode(func, node, variables)

    # We must call `getInputs()` on PcodeOpASTs
    elif type(node) == PcodeOpAST:
        nodes = list(node.getInputs())
        for node in nodes:
            if type(node.getHigh()) == HighLocal:
                variables.append(node.getHigh().getName())
            else:
                variables = get_vars_from_varnode(func, node, variables)

    return variables

# Working with Basic Blocks
Basic Blocks are collections of continuous non-branching instructions within Functions. They are joined by conditional and non-conditional branches, revealing valuable information about a program and function's code flow. This section deals with examples working with Basic Block models.

## Print details about basic blocks in a select function

In [ ]:
from ghidra.program.model.block import BasicBlockModel
from ghidra.util.task import ConsoleTaskMonitor

funcName = 'main'
blockModel = BasicBlockModel(currentProgram)
monitor = ConsoleTaskMonitor()
func = getGlobalFunctions(funcName)[0]

print("Basic block details for function '{}':".format(funcName))
blocks = blockModel.getCodeBlocksContaining(func.getBody(), monitor)

# print first block
print("\t[*] {} ".format(func.getEntryPoint()))

# print any remaining blocks
while(blocks.hasNext()):
    bb = blocks.next()
    dest = bb.getDestinations(monitor)
    while(dest.hasNext()):
        dbb = dest.next()
        # For some odd reason `getCodeBlocksContaining()` and `.next()`
        # return the root basic block after CALL instructions (x86). To filter
        # these out, we use `getFunctionAt()` which returns `None` if the address
        # is not the entry point of a function. See:
        # https://github.com/NationalSecurityAgency/ghidra/issues/855
        if not getFunctionAt(dbb.getDestinationAddress()):
            print("\t[*] {} ".format(dbb))

# Working with the Decompiler
## Print decompiled code for a specific function
Ghidra's decompiler is an exceptional resource. In certain cases you might want to extract the decompiler output for a list of functions. Here's an easy way to gather that information. Just add your own file I/O code to dump everything to individual files for analysis.

In [ ]:
from ghidra.app.decompiler import DecompInterface
from ghidra.util.task import ConsoleTaskMonitor

program = getCurrentProgram()
ifc = DecompInterface()
ifc.openProgram(program)

# here we assume there is only one function named `main`
function = getGlobalFunctions('main')[0]

# decompile the function and print the pseudo C
results = ifc.decompileFunction(function, 0, ConsoleTaskMonitor())
print(results.getDecompiledFunction().getC())

## Getting variable information from the decompiler
Ghidra's decompiler performs a lot of analysis in order to recover variable information. If you're interested in getting this information you'll need to use the decompiler interface to get a high function and its symbols. Once you have this data, you can enumerate the symbols and retrieve information about variables in the target function. Let's take a look at an example decompiled function:



In [ ]:
undefined8 func(int param_1,int param_2)
{
  long in_FS_OFFSET;
  uint auStack88 [8];
  undefined4 auStack56 [10];
  long local_10;

  local_10 = *(long *)(in_FS_OFFSET + 0x28);
  auStack56[param_1] = 1;
  printf("%d\n",(ulong)auStack88[param_2]);
  if (local_10 != *(long *)(in_FS_OFFSET + 0x28)) {
    __stack_chk_fail();
  }
  return 0;
}

The decompiled function above shows two stack variables that seem interesting to us; auStack88 and auStack56. Let's get that information programmatically.

In [ ]:
from ghidra.app.decompiler import DecompileOptions
from ghidra.app.decompiler import DecompInterface
from ghidra.util.task import ConsoleTaskMonitor

func_name = "func"
func = getGlobalFunctions(func_name)[0]
options = DecompileOptions()
monitor = ConsoleTaskMonitor()
ifc = DecompInterface()
ifc.setOptions(options)
ifc.openProgram(func.getProgram())
res = ifc.decompileFunction(func, 60, monitor)
high_func = res.getHighFunction()
lsm = high_func.getLocalSymbolMap()
symbols = lsm.getSymbols()

for i, symbol in enumerate(symbols):
    print("\nSymbol {}:".format(i+1))
    print("  name:         {}".format(symbol.name))
    print("  dataType:     {}".format(symbol.dataType))
    print("  getPCAddress: 0x{}".format(symbol.getPCAddress()))
    print("  size:         {}".format(symbol.size))
    print("  storage:      {}".format(symbol.storage))
    print("  parameter:    {}".format(symbol.parameter))
    print("  readOnly:     {}".format(symbol.readOnly))
    print("  typeLocked:   {}".format(symbol.typeLocked))
    print("  nameLocked:   {}".format(symbol.nameLocked))
    print("  slot:         {}".format(symbol.slot))

# Working with Comments
## Get all Automatic comments for a function
Ghidra adds "automatic comments" (light gray in color) in the EOL field. Here's how you can access those comments.

In [ ]:
from ghidra.app.util import DisplayableEol

listing = currentProgram.getListing()
func = getGlobalFunctions("frame_dummy")[0]
addrSet = func.getBody()
codeUnits = listing.getCodeUnits(addrSet, True)

for codeUnit in codeUnits:
	deol = DisplayableEol(codeUnit, True, True, True, True, 5, True)
	if deol.hasAutomatic():
		ac = deol.getAutomaticComment()
		print(type(ac))
		print(ac)
		print(ac[0])

# Get specific comment types for all functions
Ghidra supports 5 unique comment types users can add to their projects. This snippet shows you show to print all comments by type. This snippet is a slightly modified version of what user u/securisec posted in the Ghidra subreddit, r/ghidra. Thanks!

In [ ]:
fm = currentProgram.getFunctionManager()
listing = currentProgram.getListing()
funcs = fm.getFunctions(True) # True means iterate forward

comment_types = {
    0: 'EOL',
    1: 'PRE',
    2: 'POST',
    3: 'PLATE',
    4: 'REPEATABLE',
}

for func in funcs:
    addrSet = func.getBody()
    codeUnits = listing.getCodeUnits(addrSet, True)
    for codeUnit in codeUnits:
        for i, comment_type in comment_types.items():
            comment = codeUnit.getComment(i)
            if comment is not None:
                comment = comment.decode("utf-8")
                print("[{} : {}] {}: {}".format(func.name, codeUnit.address, comment_type, comment))

# Working with PCode
## Emulating a function
Instruction emulation is an extremely powerful technique to asist in static code analysis. Rarely however, do we have the full memory context in which dynamic code executes. So while emulation can bring an element of 'dynamic' analysis to static RE, it's typically plagued with problems of unknown memory state. For simple code this might be no problem. For object oriented code this can be a major difficulty. Either way, some element of emulation can help tremendously in speeding up analysis. Ghidra uses its internal intermediate representation (PCode) to define what instructions do. Emulation is the process of tracking these changes in a cumulative state. Unfortunely Ghidra doesn't provide fancy GUI controls around the emulator (yet), but it's fully scriptable. Ghidra v9.1 added emprovements to the EmulatorHelper class, which is really quite amazing. Here's a simple example of what you can do with it.

In [ ]:
from ghidra.app.emulator import EmulatorHelper
from ghidra.program.model.symbol import SymbolUtilities

# == Helper functions ======================================================
def getAddress(offset):
    return currentProgram.getAddressFactory().getDefaultAddressSpace().getAddress(offset)

def getSymbolAddress(symbolName):
    symbol = SymbolUtilities.getLabelOrFunctionSymbol(currentProgram, symbolName, None)
    if (symbol != None):
        return symbol.getAddress()
    else:
        raise("Failed to locate label: {}".format(symbolName))

def getProgramRegisterList(currentProgram):
    pc = currentProgram.getProgramContext()
    return pc.registers

# == Main function =========================================================
def main():
    CONTROLLED_RETURN_OFFSET = 0

    # Identify function to be emulated
    mainFunctionEntry = getSymbolAddress("main")

    # Establish emulation helper, please check out the API docs
    # for `EmulatorHelper` - there's a lot of helpful things
    # to help make architecture agnostic emulator tools.
    emuHelper = EmulatorHelper(currentProgram)

    # Set controlled return location so we can identify return from emulated function
    controlledReturnAddr = getAddress(CONTROLLED_RETURN_OFFSET)

    # Set initial RIP
    mainFunctionEntryLong = int("0x{}".format(mainFunctionEntry), 16)
    emuHelper.writeRegister(emuHelper.getPCRegister(), mainFunctionEntryLong)

    # For x86_64 `registers` contains 872 registers! You probably don't
    # want to print all of these. Just be aware, and print what you need.
    # To see all supported registers. just print `registers`.
    # We won't use this, it's just here to show you how to query
    # valid registers for your target architecture.
    registers = getProgramRegisterList(currentProgram)

    # Here's a list of all the registers we want printed after each
    # instruction. Modify this as you see fit, based on your architecture.
    reg_filter = [
        "RIP", "RAX", "RBX", "RCX", "RDX", "RSI", "RDI",
        "RSP", "RBP", "rflags"
    ]

    # Setup your desired starting state. By default, all registers
    # and memory will be 0. This may or may not be acceptable for
    # you. So please be aware.
    emuHelper.writeRegister("RAX", 0x20)
    emuHelper.writeRegister("RSP", 0x000000002FFF0000)
    emuHelper.writeRegister("RBP", 0x000000002FFF0000)

    # There are a couple of ways to write memory, use `writeMemoryValue` if you want
    # to set a small typed value (e.g. uint64). Use `writeMemory` if you're mapping in
    # a lot of memory (e.g. from a debugger memory dump). Note that each of these
    # methods write with different endianess, see the example output.
    emuHelper.writeMemoryValue(getAddress(0x000000000008C000), 4, 0x99AABBCC)  # writes big endian
    emuHelper.writeMemory(getAddress(0x00000000000CF000), b'\x99\xAA\xBB\xCC') # writes little endian

    # You can verify writes worked, or just read memory at select points
    # during emulation. Here's a couple of examples:
    mem1 = emuHelper.readMemory(getAddress(0x000000000008C000), 4)
    mem2 = emuHelper.readMemory(getAddress(0x00000000000CF000), 4)
    print("Memory at 0x000000000008C000: {}".format(mem1))
    print("Memory at 0x00000000000CF000: {}".format(mem2))

    print("Emulation starting at 0x{}".format(mainFunctionEntry))
    while monitor.isCancelled() is False:

        # Check the current address in the program counter, if it's
        # zero (our `CONTROLLED_RETURN_OFFSET` value) stop emulation.
        # Set this to whatever end target you want.
        executionAddress = emuHelper.getExecutionAddress()
        if (executionAddress == controlledReturnAddr):
            print("Emulation complete.")
            return

        # Print current instruction and the registers we care about
        print("Address: 0x{} ({})".format(executionAddress, getInstructionAt(executionAddress)))
        for reg in reg_filter:
            reg_value = emuHelper.readRegister(reg)
            print("  {} = {:#018x}".format(reg, reg_value))

        # single step emulation
        success = emuHelper.step(monitor)
        if (success == False):
            lastError = emuHelper.getLastError()
            printerr("Emulation Error: '{}'".format(lastError))
            return

    # Cleanup resources and release hold on currentProgram
    emuHelper.dispose()

# == Invoke main ===========================================================
main()

# Dumping Raw PCode
PCode exists in two primary forms you as a user should consider, "raw" and "refined". In documentation both forms are simply referred to as "PCode" making it confusing to talk about - so I distinguish between the forms using raw and refined. Just know theses are not universally accepted terms.

So raw PCode is the first pass, and the form that's displayed in the "Listing" pane inside the Ghidra UI. It's extremely verbose and explicit. This is the form you want to use when emulating, if you're writing a symbolic executor, or anything of the sort. If you want details from the decompiler passes, you want to analyze refined PCode, not this stuff! So what does it look like and how do you access it? Let's take a look.

In [ ]:
def dump_raw_pcode(func):
    func_body = func.getBody()
    listing = currentProgram.getListing()
    opiter = listing.getInstructions(func_body, True)
    while opiter.hasNext():
        op = opiter.next()
        raw_pcode = op.getPcode()
        print("{}".format(op))
        for entry in raw_pcode:
            print("  {}".format(entry))

func = getGlobalFunctions("main")[0]    # assumes only one function named `main`
dump_raw_pcode(func)            	    # dump raw pcode as strings

# Dumping Refined PCode
PCode exists in two primary forms you as a user should consider, "raw" and "refined". In documentation both forms are simply referred to as "PCode" making it confusing to talk about - so I distinguish between the forms using raw and refined. Just know theses are not universally accepted terms.

So refined PCode is heavily processed. It highly relates to the output you see in the decompiler, and if you're interested in making use of the Ghidra decompiler passes, this is the form of PCode you'll want to analyze. There are many interesting aspects of refined PCode we do not cover here, including unique values and name spaces. Just know that what might appear to be simple has a lot of analysis backing it and digging into these refined PCode elements are worth your time.

In [ ]:
from ghidra.util.task import ConsoleTaskMonitor
from ghidra.app.decompiler import DecompileOptions, DecompInterface

# == helper functions =============================================================================
def get_high_function(func):
    options = DecompileOptions()
    monitor = ConsoleTaskMonitor()
    ifc = DecompInterface()
    ifc.setOptions(options)
    ifc.openProgram(getCurrentProgram())
    # Setting a simplification style will strip useful `indirect` information.
    # Please don't use this unless you know why you're using it.
    #ifc.setSimplificationStyle("normalize")
    res = ifc.decompileFunction(func, 60, monitor)
    high = res.getHighFunction()
    return high

def dump_refined_pcode(func, high_func):
    opiter = high_func.getPcodeOps()
    while opiter.hasNext():
        op = opiter.next()
        print("{}".format(op.toString()))

# == run examples =================================================================================
func = getGlobalFunctions("main")[0]    # assumes only one function named `main`
hf = get_high_function(func)            # we need a high function from the decompiler
dump_refined_pcode(func, hf)            # dump straight refined pcode as strings

# Plotting a Function AST
Ghidra does not define a default graph provider, so you cannot graph abstract synatax trees out of the box. Here's a snippet that takes elements from Ghidra's Graph Java snippets and hacks them together to get an SVG version of a function's AST. This requires you to have GraphViz installed with the dot binary in your PATH, and networkx and pydot accessible from the Ghidra Jython console. Basically, I just copy my Python2 site-packages over to my ghidra_scripts directory and everything works. Keep in mind that if your Python module uses a natively compiled element (like Matplot lib does), you won't be able to use it in Jython. If you know of a way, please let me know.

In [ ]:
import networkx as nx
import pydot

from ghidra.app.script import GhidraScript
from ghidra.util.task import ConsoleTaskMonitor
from ghidra.app.decompiler import DecompileOptions, DecompInterface
from ghidra.program.model.pcode import PcodeOp

def buildAST(func):
    options = DecompileOptions()
    monitor = ConsoleTaskMonitor()
    ifc = DecompInterface()
    ifc.setOptions(options)
    ifc.openProgram(getCurrentProgram())
    ifc.setSimplificationStyle("normalize")
    res = ifc.decompileFunction(func, 60, monitor)
    high = res.getHighFunction()
    return high

def buildGraph(graph, func, high):
    vertices = {}
    opiter = getPcodeOpIterator(high)
    while opiter.hasNext():
        op = opiter.next()
        vert = createOpVertex(func, op)
        graph.add_node(vert)
        for i in range(0, op.getNumInputs()):
            opcode = op.getOpcode()
            if (i == 0 and (opcode == PcodeOp.LOAD or opcode == PcodeOp.STORE)):
                continue
            if (i == 1 and opcode == PcodeOp.INDIRECT):
                continue
            vn = op.getInput(i)
            if (vn != None):
                v = getVarnodeVertex(graph, vertices, vn)
                graph.add_edge(v, vert)

        outvn = op.getOutput()
        if (outvn != None):
            outv = getVarnodeVertex(graph, vertices, outvn)
            if (outv != None):
                graph.add_edge(vert, outv)

def createOpVertex(func, op):
    name = op.getMnemonic()
    id = getOpKey(op)
    opcode = op.getOpcode()
    if ((opcode == PcodeOp.LOAD) or (opcode == PcodeOp.STORE)):
        vn = op.getInput(0)
        addrspace = currentProgram.getAddressFactory().getAddressSpace(vn.getOffset())
        name += ' ' + addrspace.getName()
    elif (opcode == PcodeOp.INDIRECT):
        vn = op.getInput(1)
        if (vn != None):
            indOp = high.getOpRef(vn.getOffset())
            if (indOp != None):
                name += " (" + indOp.getMnemonic() + ")"
    return "{}_{}".format(name, id)

def createVarnodeVertex(graph, vn):
    name = str(vn.getAddress())
    id = getVarnodeKey(vn)
    if (vn.isRegister()):
        reg = currentProgram.getRegister(vn.getAddress(), vn.getSize())
        if (reg != None):
            name = reg.getName()
    return "{}_{}".format(name, id)

def getVarnodeVertex(graph, vertices, vn):
    res = None
    try:
        res = vertices[str(vn.getUniqueId())]
    except KeyError:
        res = None
    if (res == None):
        res = createVarnodeVertex(graph, vn)
        vertices[str(vn.getUniqueId())] = res
    return res

def getAddress(offset):
    return currentProgram.getAddressFactory().getDefaultAddressSpace().getAddress(offset)

def getOpKey(op):
    sq = op.getSeqnum()
    id = str(sq.getTarget()) + " o " + str(op.getSeqnum().getTime())
    return id

def getPcodeOpIterator(high):
    return high.getPcodeOps()

def getVarnodeKey(vn):
    op = vn.getDef()
    id = ""
    if (op != None):
        id = str(op.getSeqnum().getTarget()) + " v " + str(vn.getUniqueId())
    else:
        id = "i v " + str(vn.getUniqueId())
    return id

def main():
    graph = nx.DiGraph()
    listing = currentProgram.getListing()
    func = getFunctionContaining(getAddress(0x00100690))
    high = buildAST(func)
    buildGraph(graph, func, high)

    dot_data = nx.nx_pydot.to_pydot(graph)
    svg = pydot.graph_from_dot_data(dot_data.to_string())[0].create_svg()
    svg_path = "C:\\Users\\username\\Desktop\\test.svg"
    f = open(svg_path, 'w')
    f.write(svg)
    f.close()

    print("Wrote pydot SVG of graph to: {}\nNodes: {}, Edges: {}".format(svg_path, len(graph.nodes), len(graph.edges)))

main()

# Working with Graphs
## Creating a Call Graph
Ghidra's complex API allows for the creation of various graph structures including directional graphs (digraphs). This example shows how to create a DiGraph of vertices (functions) and edges (calls from/to).

Note that adding a vertex or an edge between two vertex entries does not reuse or override them! This is because, while many nodes share the same name, they contain unique hash codes (keys). If you were looking to trim this graph to include only unqiue nodes, you would need to consider both the name of the symbol and its address to account for overridden functions.

In its current form, this DiGraph is unlikely to be of any use to you. But the building blocks of creating interesting control flow graphs (CFG), program dependence graphs (PDG), data dependency graphs (DDG), and other graphs are all here.

In [ ]:
from ghidra.util.graph import DirectedGraph
from ghidra.util.graph import Edge
from ghidra.util.graph import Vertex

def getAddress(offset):
    return currentProgram.getAddressFactory().getDefaultAddressSpace().getAddress(offset)

digraph = DirectedGraph()
listing = currentProgram.getListing()
fm = currentProgram.getFunctionManager()

funcs = fm.getFunctions(True) # True mean iterate forward
for func in funcs:
	# Add function vertices
	print("Function: {} @ 0x{}".format(func.getName(), func.getEntryPoint())) # FunctionDB
	digraph.add(Vertex(func))

	# Add edges for static calls
	entryPoint = func.getEntryPoint()
	instructions = listing.getInstructions(entryPoint, True)
	for instruction in instructions:
		addr = instruction.getAddress()
		oper = instruction.getMnemonicString()
		if oper == "CALL":
			print("    0x{} : {}".format(addr, instruction))
			flows = instruction.getFlows()
			if len(flows) == 1:
				target_addr = "0x{}".format(flows[0])
				digraph.add(Edge(Vertex(func), Vertex(fm.getFunctionAt(getAddress(target_addr)))))

print("DiGraph info:")
edges = digraph.edgeIterator()
while edges.hasNext():
	edge = edges.next()
	from_vertex = edge.from()
	to_vertex = edge.to()
	print("  Edge from {} to {}".format(from_vertex, to_vertex))

vertices = digraph.vertexIterator()
while vertices.hasNext():
	vertex = vertices.next()
	print("  Vertex: {} (key: {})".format(vertex, vertex.key()))
    # some extra stuff you might want to see
	#print("    type(vertex):      {}".format(type(vertex)))
	#print("    vertex.hashCode(): {}".format(vertex.hashCode()))
	#print("    vertex.referent(): {}".format(vertex.referent()))
	#print("    type(referent):    {}".format(type(vertex.referent())))

# Miscellaneous
## Program Slices
Given a specific varnode, Ghidra is able to generate backward and forward program slices. This functionality is contained in the DecompilerUtils class as four functions; getBackwardSlice, getForwardSlice, getForwardSliceToPCodeOps, and getBackwardSliceToPCodeOps.

You can read more about program slicing online, but the idea is to focus on a specific varnode (say a variable) and slice away anything unrelated to it so you're left with a slice of program related to the element you're interested in. In the Ghidra UI you can right-click a variable and select a program slice option and Ghidra will highlight the slice. If you right-click and don't see an option, you're not clicking on a compatable varnode / element, try something else.

In [ ]:
from ghidra.util.task import ConsoleTaskMonitor
from ghidra.app.decompiler import DecompileOptions, DecompInterface
from ghidra.app.decompiler.component import DecompilerUtils

# == helper functions =============================================================================
def get_high_function(func):
    options = DecompileOptions()
    monitor = ConsoleTaskMonitor()
    ifc = DecompInterface()
    ifc.setOptions(options)
    ifc.openProgram(getCurrentProgram())
    res = ifc.decompileFunction(func, 60, monitor)
    high = res.getHighFunction()
    return high

# == run examples =================================================================================
func = getGlobalFunctions("main")[0]
hf = get_high_function(func)
lsm = hf.getLocalSymbolMap()
symbols = lsm.getSymbols()

for symbol in symbols:
    print("\nSymbol name: {}".format(symbol.getName()))
    hv = symbol.getHighVariable()
    # Try this snippet with `hv.getInstances()` to enumerate slices for all instances!
    varnode = hv.getRepresentative()
    print("Varnode: {}".format(varnode))
    fs = DecompilerUtils.getForwardSlice(varnode)
    print("Forward Slice: {}".format(fs))
    bs = DecompilerUtils.getBackwardSlice(varnode)
    print("Backward Slice: {}".format(bs))
    bswo = DecompilerUtils.getBackwardSliceToPCodeOps(varnode)
    print("Backward Slice w/ PCode Ops: {}".format(bswo))
    fswo = DecompilerUtils.getForwardSliceToPCodeOps(varnode)
    print("Forward Slice w/ PCode Ops: {}".format(fswo))